In [122]:
import requests
from bs4 import BeautifulSoup

# Mine list of contests
index_link = "https://www.topcoder.com/tc?module=MatchList&sc=&sd=&nr=200&sr={}"
round_ids = []
for page_id in range(1, 1300, 200):
    url = index_link.format(page_id)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    results = soup.find_all("td", {"class": "value"})
    for result in results:
        ref = result.find('a', href=True)['href']
        round_id = int(ref.split('=')[-1])
        round_ids.append(round_id)

[18301, 18300, 18298, 18287, 18286, 18285, 18284, 18280, 18279, 18278, 18267, 18266, 18261, 18262, 18259, 18258, 18197, 18194, 18192, 18191, 18188, 18146, 18145, 18141, 18139, 18138, 18117, 18116, 18114, 18111, 18085, 18081, 18080, 18079, 18078, 18077, 18075, 18074, 18069, 18068, 17994, 17989, 17965, 17963, 17962, 17958, 17907, 17906, 17903, 17900, 17857, 17853, 17851, 17848, 17805, 17802, 17799, 17776, 17773, 17770, 17746, 17743, 17741, 17738, 17726, 17723, 17720, 17687, 17684, 17681, 17678, 17666, 17665, 17663, 17660, 17659, 17658, 17637, 17633, 17632, 17631, 17628, 17617, 17614, 17612, 17611, 17608, 17573, 17592, 17590, 17589, 17574, 17570, 17569, 17536, 17535, 17534, 17531, 17529, 17528, 17512, 17514, 17510, 17509, 17496, 17493, 17488, 17457, 17456, 17450, 17422, 17426, 17420, 17400, 17399, 17398, 17397, 17396, 17395, 17392, 17373, 17370, 17354, 17351, 17350, 17348, 17316, 17310, 17298, 17270, 17265, 17259, 17247, 17246, 17242, 17239, 17238, 17228, 17226, 17221, 17220, 17219, 17206

In [98]:


magic_link = "https://community.topcoder.com/stat?c=round_stats&rd={}&sm=1&em=100&nm=99&dn={}"
def get_round_html(round_id, div):
    url = magic_link.format(round_id, div)
    html = requests.get(url)
    return html.text
    
def reconstruct_rankings(round_id, div):    
    text = get_round_html(round_id, div)
    soup = BeautifulSoup(text, 'html.parser')
    # Lines with statText are the ones with ranks
    results = soup.find_all("td", {"class": "statText"})
    decoded_text = [res.get_text(strip=True) for res in results]
    # First 20 lines are junk
    decoded_text = decoded_text[19:]
    
    # Every 18 lines is a new participant
    def parse_rank(text):
        name = text[1]
        points = float(text[8])
        return (points, name)
    ranks = []
    i = 0
    while i + 18 <= len(decoded_text):
        if 'Room' in decoded_text[i]:
            i += 1
            continue
            
        try:
            ranks.append(parse_rank(decoded_text[i:i+18]))
        except:
            pass
        i += 18
        
    return sorted(ranks, reverse=True)

In [125]:
contests = []
for round_id in round_ids:
    for div in range(2):
        ranks = reconstruct_rankings(round_id, div+1)
        if ranks:
            contests.append(ranks)
    print('Completed round id', round_id)

Completed round id 18301
Completed round id 18300
Completed round id 18298
Completed round id 18287
Completed round id 18286
Completed round id 18285
Completed round id 18284
Completed round id 18280
Completed round id 18279
Completed round id 18278
Completed round id 18267
Completed round id 18266
Completed round id 18261
Completed round id 18262
Completed round id 18259
Completed round id 18258
Completed round id 18197
Completed round id 18194
Completed round id 18192
Completed round id 18191
Completed round id 18188
Completed round id 18146
Completed round id 18145
Completed round id 18141
Completed round id 18139
Completed round id 18138
Completed round id 18117
Completed round id 18116
Completed round id 18114
Completed round id 18111
Completed round id 18085
Completed round id 18081
Completed round id 18080
Completed round id 18079
Completed round id 18078
Completed round id 18077
Completed round id 18075
Completed round id 18074
Completed round id 18069
Completed round id 18068


In [140]:
import json
from collections import defaultdict

for r, contest in enumerate(contests):
    data = {}
    data['id'] = r
    data['name'] = "To be filled."
    data['time_seconds'] = 0
    
    standings = []
    data['standings'] = standings
    i = 0
    while i < len(contest):
        j = 0
        while i + j < len(contest) and contest[i + j][0] == contest[i][0]:
            j += 1
        
        for k in range(j):
            standings.append([contest[i+k][1], i, i + j - 1])
        i += j
        
    ranking_file = open('../cache/topcoder/{}.json'.format(r), 'w')
    json.dump(data, ranking_file)
    ranking_file.close()
    
contest_ids = list(range(len(contests)))
with open('../data/topcoder/contest_ids.json', 'w') as out:
    out.write(str(contest_ids))
    out.close()